In [2]:
# !pip install -U sentence-transformers

  Obtaining dependency information for sentence-transformers from https://files.pythonhosted.org/packages/58/4b/922436953394e1bfda05e4bf1fe0e80f609770f256c59a9df7a9254f3e0d/sentence_transformers-3.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for transformers<5.0.0,>=4.34.0 from https://files.pythonhosted.org/packages/75/35/07c9879163b603f0e464b0f6e6e628a2340cfc7cdc5ca8e7d52d776710d4/transformers-4.44.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.7 kB ? eta -:--:--
     -------------------------- ----------- 30.7/43.7 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 43.7/43.7 kB 428.4 kB/s eta 0:00:00
  Obtaining dependency information for huggingface-hub>=0.15.1 from https://files.pythonhosted.org/packages/b9/8f/d6718641c14d98a5848c6a24d2376028d292074ffade0702940a4b1dde76/huggingface_hub-0.24.6-py3-none-any.whl.metadata
  Obtaining depende

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2024.6.1 which is incompatible.


In [1]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)


C:\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[[ 0.02250258 -0.07829181 -0.02303076 ... -0.00827928  0.02652692
  -0.00201897]
 [ 0.04170238  0.0010974  -0.01553418 ... -0.02181626 -0.0635936
  -0.00875283]]


In [2]:
print(len(embeddings[0]))

768


In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted', 'sentence is example that each word takes id']
# sentences = ['This is an example sentence', 'Each sentence is converted']
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

C:\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Abdelrahman Elewah\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

C:\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Sentence embeddings:
tensor([[ 0.0225, -0.0783, -0.0230,  ..., -0.0083,  0.0265, -0.0020],
        [ 0.0417,  0.0011, -0.0155,  ..., -0.0218, -0.0636, -0.0088],
        [ 0.0416, -0.0073, -0.0287,  ..., -0.0012,  0.0058,  0.0316]])


In [2]:
len(sentence_embeddings[0])

768

In [3]:
for i in encoded_input:
    print(i)

input_ids
attention_mask


In [4]:
type(encoded_input)

transformers.tokenization_utils_base.BatchEncoding

In [5]:
encoded_input["input_ids"]

tensor([[   0, 2027, 2007, 2023, 2746, 6255,    2,    1,    1,    1],
        [   0, 2173, 6255, 2007, 4995,    2,    1,    1,    1,    1],
        [   0, 6255, 2007, 2746, 2012, 2173, 2777, 3142, 8913,    2]])

In [6]:
encoded_input["attention_mask"]

tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [7]:
from docarray import BaseDoc
from docarray.typing import NdArray

class ToyDoc(BaseDoc):
    text: str = ''
    embedding: NdArray[128]

In [8]:
from docarray import DocList
import numpy as np
from vectordb import InMemoryExactNNVectorDB, HNSWVectorDB

# Specify your workspace path
db = InMemoryExactNNVectorDB[ToyDoc](workspace='./workspace_path')

# Index a list of documents with random embeddings
doc_list = [ToyDoc(text=f'toy doc {i}', embedding=np.random.rand(128)) for i in range(1000)]
db.index(inputs=DocList[ToyDoc](doc_list))

# Perform a search query
query = ToyDoc(text='50', embedding=np.random.rand(128))
results = db.search(inputs=DocList[ToyDoc]([query]), limit=2)

print(query.text)
# Print out the matches
for m in results[0].matches:
    print(m)
    

WARNING - docarray - Index file does not exist: G:\My Drive\ontario tech era\python-repo\SensorsConnect\langChain-Course\documentations-tutorials\VectorDB\workspace_path\InMemoryExactNNIndexer[ToyDoc][ToyDocWithMatchesAndScores]/index.bin. Initializing empty InMemoryExactNNIndex.


50


ToyDoc(
    id='cf878d4f70d130f722117b95c9514f31',
    text='toy doc 596',
    embedding=NdArray([0.36303533, 0.10351704, 0.6127303 , 0.88562399, 0.45163972,
         0.52189729, 0.44147967, 0.95455978, 0.84453813, 0.48048236,
         0.2122873 , 0.8658788 , 0.20969576, 0.94723943, 0.34405264,
         0.31180417, 0.12566114, 0.29034632, 0.34571828, 0.71184984,
         0.11974517, 0.94427399, 0.12388932, 0.92973246, 0.71154235,
         0.71410255, 0.01816357, 0.81373274, 0.39355815, 0.32184818,
         0.38007508, 0.46647514, 0.79461604, 0.84307049, 0.43097007,
         0.4198746 , 0.02910279, 0.99744866, 0.50146726, 0.80918319,
         0.94658632, 0.49910158, 0.79140675, 0.95051794, 0.33912101,
         0.69168492, 0.59947871, 0.38238263, 0.76517805, 0.38602666,
         0.59903189, 0.94523261, 0.41767843, 0.36770561, 0.7817798 ,
         0.42114472, 0.37204277, 0.51144458, 0.42528325, 0.44713802,
         0.22857368, 0.03030275, 0.60664197, 0.0115612 , 0.79386439,
         0.861

ToyDoc(
    id='7385e71f421472ffb74d329be285c647',
    text='toy doc 78',
    embedding=NdArray([0.36694325, 0.93628475, 0.37912976, 0.54062732, 0.24778555,
         0.44212224, 0.81100654, 0.71777236, 0.99613619, 0.1600981 ,
         0.06610302, 0.923664  , 0.17138439, 0.49892291, 0.56573618,
         0.52578658, 0.76336119, 0.0814125 , 0.18921653, 0.81872084,
         0.09059606, 0.14969811, 0.59854249, 0.86045097, 0.73616459,
         0.0955524 , 0.44401599, 0.52847905, 0.3978549 , 0.9500598 ,
         0.77217496, 0.31821644, 0.74795639, 0.79358018, 0.51136635,
         0.56204903, 0.75564182, 0.23021474, 0.92213383, 0.55380821,
         0.81717688, 0.52638513, 0.62093478, 0.58350444, 0.36245817,
         0.27040902, 0.68051034, 0.11059093, 0.03827634, 0.02175307,
         0.57172196, 0.80015008, 0.18433066, 0.89478791, 0.32205157,
         0.49539747, 0.98760441, 0.91509415, 0.8435242 , 0.30397629,
         0.15268248, 0.68262153, 0.0731192 , 0.60404524, 0.21414682,
         0.9683

In [34]:
sentence_embeddings

tensor([[ 0.0225, -0.0783, -0.0230,  ..., -0.0083,  0.0265, -0.0020],
        [ 0.0417,  0.0011, -0.0155,  ..., -0.0218, -0.0636, -0.0088],
        [ 0.0416, -0.0073, -0.0287,  ..., -0.0012,  0.0058,  0.0316]])

In [39]:
for i in sentence_embeddings:
    print(i)
    print(5*"**")

tensor([ 2.2503e-02, -7.8292e-02, -2.3031e-02, -5.1000e-03, -8.0340e-02,
         3.9132e-02,  1.1343e-02,  3.4648e-03, -2.9457e-02, -1.8893e-02,
         9.4743e-02,  2.9275e-02,  3.9486e-02, -4.6317e-02,  2.5425e-02,
        -3.2200e-02,  6.2193e-02,  1.5559e-02, -4.6780e-02,  5.0390e-02,
         1.4611e-02,  2.3141e-02,  1.2207e-02,  2.5070e-02,  2.9365e-03,
        -4.1982e-02, -4.0103e-03, -2.2784e-02, -7.6860e-03, -3.3109e-02,
         3.2212e-02, -2.0999e-02,  1.1673e-02, -9.8507e-02,  1.7793e-06,
        -2.2993e-02, -1.3114e-02, -2.8022e-02, -6.9997e-02,  2.5931e-02,
        -2.8950e-02,  8.7634e-02, -1.2092e-02,  3.9861e-02, -3.3138e-02,
         3.5911e-02,  3.4610e-02,  6.4978e-02, -3.0082e-02,  6.9819e-02,
        -3.9952e-03, -1.0160e-03, -3.5018e-02, -4.3657e-02,  5.0803e-02,
         4.6876e-02,  5.3966e-02, -4.0301e-02,  3.2014e-03,  1.3662e-02,
         3.8219e-02, -3.2384e-03, -7.8460e-04, -1.7119e-02,  6.9044e-03,
        -1.0924e-02,  8.6331e-03, -1.8236e-02,  1.8

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted', 'sentence is example that each word takes id']
# sentences = ['This is an example sentence', 'Each sentence is converted']
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')


model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
print("tokenizer output")

for i in encoded_input:
    print(i)
    print(encoded_input[i])
print(50*"*")

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

print("Compute token embeddings")
print(model_output)
print(50*"*")

    
# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])


print("Perform pooling")
print(sentence_embeddings)
print(50*"*")

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


C:\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tokenizer output
input_ids
tensor([[   0, 2027, 2007, 2023, 2746, 6255,    2,    1,    1,    1],
        [   0, 2173, 6255, 2007, 4995,    2,    1,    1,    1,    1],
        [   0, 6255, 2007, 2746, 2012, 2173, 2777, 3142, 8913,    2]])
attention_mask
tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
**************************************************
Compute token embeddings
BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 9.6638e-02, -1.7067e-01,  7.6037e-03,  ...,  2.3386e-02,
           1.0138e-01, -4.2516e-02],
         [ 6.8078e-02, -1.5394e-01, -1.2517e-01,  ..., -1.1160e-02,
           1.1047e-02, -1.3179e-03],
         [ 1.5145e-02, -3.7817e-01, -1.0184e-01,  ..., -8.7419e-02,
           1.0200e-01,  7.7285e-02],
         ...,
         [ 5.1867e-02,  2.4321e-01, -5.4224e-02,  ...,  8.1914e-02,
           6.1595e-02, -2.0032e-02],
         [ 5.6142e-02,  1.7018e-01, -3.4386e-02,  ...,  7.2115e-02,
   

In [4]:
encoded_input["input_ids"]

tensor([[   0, 2027, 2007, 2023, 2746, 6255,    2]])

In [7]:
encoded_input["attention_mask"]

tensor([[1, 1, 1, 1, 1, 1, 1]])

In [16]:
for i in model_output:
    print(i)
    print(model_output[i])

last_hidden_state
tensor([[[ 0.0966, -0.1707,  0.0076,  ...,  0.0234,  0.1014, -0.0425],
         [ 0.0681, -0.1539, -0.1252,  ..., -0.0112,  0.0110, -0.0013],
         [ 0.0151, -0.3782, -0.1018,  ..., -0.0874,  0.1020,  0.0773],
         ...,
         [ 0.1024, -0.3481, -0.0845,  ..., -0.1105,  0.0856, -0.0464],
         [-0.0254,  0.0078, -0.0441,  ...,  0.0958, -0.0618, -0.0061],
         [ 0.1180, -0.1216,  0.0249,  ..., -0.0096,  0.1404, -0.0325]]])
pooler_output
tensor([[ 8.7998e-02, -4.1801e-02,  1.8152e-02, -8.9510e-02, -3.9929e-02,
         -3.1443e-02,  2.3688e-02,  3.0801e-02, -8.9669e-04, -8.9895e-02,
          6.9226e-02, -1.6272e-03, -2.9426e-02,  4.3844e-02, -5.7402e-02,
          4.1095e-02, -7.4796e-02,  1.7036e-02, -1.2585e-01,  5.3080e-02,
         -8.2184e-02, -8.5882e-02, -5.2761e-02,  8.2555e-03,  1.0675e-01,
         -6.7143e-02,  1.1777e-02, -3.0840e-02,  8.3433e-02,  7.9797e-02,
         -4.2842e-02,  4.4696e-03,  5.0861e-02,  1.0116e-02, -3.6652e-02,
        

In [21]:
len((model_output["last_hidden_state"])[0])

7

In [24]:
len(model_output["last_hidden_state"][0][0])

768

In [26]:
model_output["last_hidden_state"]

tensor([[[ 0.0966, -0.1707,  0.0076,  ...,  0.0234,  0.1014, -0.0425],
         [ 0.0681, -0.1539, -0.1252,  ..., -0.0112,  0.0110, -0.0013],
         [ 0.0151, -0.3782, -0.1018,  ..., -0.0874,  0.1020,  0.0773],
         ...,
         [ 0.1024, -0.3481, -0.0845,  ..., -0.1105,  0.0856, -0.0464],
         [-0.0254,  0.0078, -0.0441,  ...,  0.0958, -0.0618, -0.0061],
         [ 0.1180, -0.1216,  0.0249,  ..., -0.0096,  0.1404, -0.0325]]])

In [29]:
len(model_output["pooler_output"][0])

768

In [30]:
model_output[0]

tensor([[[ 0.0966, -0.1707,  0.0076,  ...,  0.0234,  0.1014, -0.0425],
         [ 0.0681, -0.1539, -0.1252,  ..., -0.0112,  0.0110, -0.0013],
         [ 0.0151, -0.3782, -0.1018,  ..., -0.0874,  0.1020,  0.0773],
         ...,
         [ 0.1024, -0.3481, -0.0845,  ..., -0.1105,  0.0856, -0.0464],
         [-0.0254,  0.0078, -0.0441,  ...,  0.0958, -0.0618, -0.0061],
         [ 0.1180, -0.1216,  0.0249,  ..., -0.0096,  0.1404, -0.0325]]])

In [10]:
*encoded_input

SyntaxError: can't use starred expression here (762731624.py, line 1)

In [5]:
encoded_input

{'input_ids': tensor([[   0, 2027, 2007, 2023, 2746, 6255,    2,    1,    1,    1],
        [   0, 2173, 6255, 2007, 4995,    2,    1,    1,    1,    1],
        [   0, 6255, 2007, 2746, 2012, 2173, 2777, 3142, 8913,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
for i in **encoded_input:
    print(i)

SyntaxError: invalid syntax (2899493380.py, line 1)